In [1]:
import csv
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import pandas
import pandas as pd
from tqdm import tqdm

In [2]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

/Users/qhan/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/qhan/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/qhan/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/qhan/.pyenv/versions/3.7.4/lib/python3.7/s

In [3]:
%%time

ws = WS("../data")
pos = POS("../data")
ner = NER("../data")

CPU times: user 14.7 s, sys: 4.91 s, total: 19.6 s
Wall time: 20.6 s


In [4]:
%%time

df = pd.read_csv("../data/article_full.csv", encoding="utf-8", index_col="news_ID")

CPU times: user 195 ms, sys: 48.6 ms, total: 244 ms
Wall time: 250 ms


In [5]:
%%time

skipped_indices = []

for i, row in df.iterrows():
    try:
        raw_content = row['content_full']
        tmp_content = bs(raw_content, 'html.parser').text
        tmp_content = ''.join(tmp_content.split())
        clean_content = tmp_content.replace('\n','').replace('\r','').replace('\xa0','').replace('\t','').replace('\u3000','').replace(' ','')
        row['content_full'] = clean_content
        
    except TypeError:
        skipped_indices.append(i)
        row['content_full'] = "."

CPU times: user 1.37 s, sys: 14.6 ms, total: 1.38 s
Wall time: 1.39 s


In [6]:
df.to_csv('../data/article_qhan.csv', encoding='utf-8')

In [18]:
%%time
sentence_list = list(df['content_full'])

CPU times: user 1.11 ms, sys: 21 µs, total: 1.13 ms
Wall time: 1.16 ms


In [19]:
%%time
word_sentence_list = ws(sentence_list)

CPU times: user 3h 11min 21s, sys: 15min 40s, total: 3h 27min 1s
Wall time: 37min 12s


In [20]:
%%time
pos_sentence_list = pos(word_sentence_list)

CPU times: user 1h 45min 9s, sys: 13min 6s, total: 1h 58min 15s
Wall time: 18min 49s


In [21]:
%%time
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

CPU times: user 3h 39min 16s, sys: 16min 6s, total: 3h 55min 23s
Wall time: 42min 26s


In [33]:
entity_sentence_list_2 = [list(item) for item in entity_sentence_list]

In [36]:
import json
with open("../data/word-sentence.json", 'w', encoding='utf8') as f:
    json.dump(word_sentence_list, f, indent=4, ensure_ascii=False)
    
with open("../data/pos-sentence.json", 'w', encoding='utf8') as f:
    json.dump(pos_sentence_list, f, indent=4, ensure_ascii=False)
    
with open("../data/entity-sentence.json", 'w', encoding='utf8') as f:
    json.dump(entity_sentence_list_2, f, indent=4, ensure_ascii=False)
    